# Training of CNN with 1x1 filter

Zašto korištenje 1x1 filtera?
Zato da algoritam pronađe vezu između true ishoda i dubine ulaza.

Planiram koristiti:
- INICIJALIZACIJA??
- aktivacijska funkcija: Relu
- normalizacija: Batch Normalization - zapravo, s obzirom da je Batch norm. koristan prvenstveno kod dubokih mreža, pokušati ću prvo bez toga, da vidim kako funkcionira.
- regularizacija: Dropout
- OPTIMIZER? ADAM?

S obzirom da ima malo podataka, razmisli može li se nekako povećati broj podataka data augmentation tehnikama.

In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
import csv

from sklearn.metrics import accuracy_score

# Common imports
import numpy as np
import os

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


%matplotlib inline

In [2]:
print ('DATA PREPARATION')
vege_csv = csv.reader(open('vegetacija.csv', newline=''), delimiter=' ', quotechar='|')

vege_array = []


for row in vege_csv:
	red_array = []
	items = row[0].split(';')
	for i in items:
		red_array.append(i)

	vege_array.append(red_array)


klase = []
values = []

vege_array_fix = vege_array[:-4]

print ('length of the fixed array is ', len (vege_array_fix))

for r in vege_array_fix:
    value_row = []
    i = 0
    while i < len(r):
        if i == 0:
            if r[i] == 'X':
                klase.append(0)
            if r[i] == 'V':
                klase.append(1)
            if r[i] == 'N':
                klase.append(2)
        else:
            value_row.append(int(r[i]))
        i = i + 1
    values.append(value_row)

print ('test : ', vege_array[10000])

DATA PREPARATION
length of the fixed array is  199400
test :  ['V', '339', '201', '238', '278', '186', '298', '298', '253']


In [3]:
X_train, X_test, y_train, y_test = train_test_split(values, klase, test_size=0.20)

In [4]:
reset_graph()

In [5]:
height = 1
width = 1
channels = 8
# n_inputs = height * width

conv1_fmaps = 16
conv1_ksize = 1
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 32
conv2_ksize = 1
conv2_stride = 1
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 3

# n_inputs = len(X_train)

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, channels], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")    


with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 1, 1, 1], strides=[1, 1, 1, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 1 * 1])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [6]:
print (len(X_train))

159520


In [7]:
X_valid = X_train[:63808]
X_train = X_train[63808:]

y_valid = y_train[:63808]
y_train = y_train[63808:]

In [8]:
train_array_length = len(X_train)
print (train_array_length)

95712


In [9]:
#moja egzekucijska funkcija

# n_epochs = 100
# batch_size = 20

# with tf.Session() as sess:
#     init.run()
#     for epoch in range(n_epochs):
#         for iteration in range(int(len(X_train)/batch_size)):
#             X_batch, y_batch = X_train.train.next_batch(batch_size)
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#         loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
#         if loss_val < best_loss:
#             save_path = saver.save(sess, "./vege_model.ckpt")
#             best_loss = loss_val
#             checks_without_progress = 0
#         else:
#             checks_without_progress += 1
#             if checks_without_progress > max_checks_without_progress:
#                 print("Early stopping!")
#                 break
#         print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
#             epoch, loss_val, best_loss, acc_val * 100))

# with tf.Session() as sess:
#     saver.restore(sess, "./vege_model.ckpt")
#     acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
#     print("Final test accuracy: {:.2f}%".format(acc_test * 100))

In [10]:

# n_epochs = 10
# batch_size = 20

# max_checks_without_progress = 20
# checks_without_progress = 0
# best_loss = np.infty

# with tf.Session() as sess:
#     init.run()

#     for epoch in range(n_epochs):
#         rnd_idx = np.random.permutation(train_array_length)
#         for rnd_indices in np.array_split(rnd_idx, train_array_length // batch_size):
#             print (rnd_indices)

In [11]:

n_epochs = 100
batch_size = 40

max_checks_without_progress = 40
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch = []
            y_batch = []
            for i in rnd_indices:
                X_batch.append(X_train[i]) 
                y_batch.append(y_train[i])
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./vege_model.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./vege_model.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.613481	Best loss: 0.613481	Accuracy: 81.70%
1	Validation loss: 0.578504	Best loss: 0.578504	Accuracy: 80.76%
2	Validation loss: 0.568325	Best loss: 0.568325	Accuracy: 81.74%
3	Validation loss: 0.562645	Best loss: 0.562645	Accuracy: 81.80%
4	Validation loss: 0.561915	Best loss: 0.561915	Accuracy: 81.86%
5	Validation loss: 0.572783	Best loss: 0.561915	Accuracy: 81.73%
6	Validation loss: 0.561641	Best loss: 0.561641	Accuracy: 81.80%
7	Validation loss: 0.557554	Best loss: 0.557554	Accuracy: 81.71%
8	Validation loss: 0.552568	Best loss: 0.552568	Accuracy: 81.84%
9	Validation loss: 0.545297	Best loss: 0.545297	Accuracy: 81.82%
10	Validation loss: 0.544298	Best loss: 0.544298	Accuracy: 81.74%
11	Validation loss: 0.547756	Best loss: 0.544298	Accuracy: 81.73%
12	Validation loss: 0.547996	Best loss: 0.544298	Accuracy: 81.85%
13	Validation loss: 0.550838	Best loss: 0.544298	Accuracy: 81.79%
14	Validation loss: 0.542863	Best loss: 0.542863	Accuracy: 81.82%
15	Validation loss: 